In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [47]:
train = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv', index_col=['id'])
test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col=['id'])

train

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312
...,...,...,...,...,...,...,...,...,...,...
299995,Adidas,Leather,Small,9.0,No,No,Tote,Blue,12.730812,129.99749
299996,Jansport,Leather,Large,6.0,No,Yes,Tote,Blue,26.633182,19.85819
299997,Puma,Canvas,Large,9.0,Yes,Yes,Backpack,Pink,11.898250,111.41364


In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 290295 non-null  object 
 1   Material              291653 non-null  object 
 2   Size                  293405 non-null  object 
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    292556 non-null  object 
 5   Waterproof            292950 non-null  object 
 6   Style                 292030 non-null  object 
 7   Color                 290050 non-null  object 
 8   Weight Capacity (kg)  299862 non-null  float64
 9   Price                 300000 non-null  float64
dtypes: float64(3), object(7)
memory usage: 25.2+ MB


*Create a pairplot*

*Get numeric columns and categorical columns*

In [49]:
num_df = train.select_dtypes(include=['number']).drop('Price', axis=1)
cat_df = train.select_dtypes(include=['object'])

num_cols = num_df.columns
cat_cols = cat_df.columns

num_df

,Compartments,Weight Capacity (kg)
id,,
0,7.0,11.611723
1,10.0,27.078537
2,2.0,16.643760
3,8.0,12.937220
4,1.0,17.749338
...,...,...
299995,9.0,12.730812
299996,6.0,26.633182
299997,9.0,11.898250


*Seperate categorical columns into ordinal (to be ordinal encoded) and non-ordinal (to be one-hot encoded)*

# TODO: Try changing the ordinal and non-ordinal columns around

In [50]:
ord_cols = ['Size', 'Laptop Compartment', 'Waterproof']
oh_cols = ['Brand', 'Material', 'Style', 'Color']

ord_df = train[ord_cols]
oh_df = train[oh_cols]

*Get unique values*

In [51]:
for col in ord_df.columns:
    print(f'col: {col}\n\
values: {ord_df[col].unique()}')

col: Size
values: ['Medium' 'Small' 'Large' nan]
col: Laptop Compartment
values: ['Yes' 'No' nan]
col: Waterproof
values: ['No' 'Yes' nan]


*Get the ordering of columns in the `train` DataFrame*

In [52]:
for i, col in enumerate(train.columns):
    print(f'{i}, {col}')

0, Brand
1, Material
2, Size
3, Compartments
4, Laptop Compartment
5, Waterproof
6, Style
7, Color
8, Weight Capacity (kg)
9, Price


 *Define ordering for each value*

In [53]:
size_categories = ['Small', 'Medium', 'Large']
yesno_categories = ['No', 'Yes']

*Create transformers for each type of column*

In [54]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer

# Numerical columns

num_transformer = Pipeline(
    steps=[
        ('num_impute', SimpleImputer(strategy='median')),
        ('scale', StandardScaler()),
    ]
)

num_df = pd.DataFrame(num_transformer.fit_transform(num_df))
num_df.columns = num_cols

# For ordinal columns

ord_preprocessor = ColumnTransformer(
    transformers=[
        ('size_encoder', OrdinalEncoder(categories=[size_categories], ), ['Size']),
        ('laptop_encoder', OrdinalEncoder(categories=[yesno_categories]), ['Laptop Compartment']),
        ('wp_encoder', OrdinalEncoder(categories=[yesno_categories]), ['Waterproof'])
    ],
    remainder='passthrough'
)

ord_transformer = Pipeline(
    steps=[
        ('ord_impute', SimpleImputer(strategy='most_frequent')),
        ('df_conversion', FunctionTransformer(lambda x: pd.DataFrame(x, columns=ord_cols))),
        ('preprocess', ordinal_preprocessor)
    ]
)

ord_df = pd.DataFrame(ord_transformer.fit_transform(ord_df), columns = ord_cols)

# One hot columns

oh_transformer = Pipeline(
    steps=[
        ('oh_impute', SimpleImputer(strategy='most_frequent')),
        ('oh_encode', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ]
)

oh_df = pd.DataFrame(oh_transformer.fit_transform(oh_df))



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [62]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

my_model = xgb.XGBRegressor(objective='reg:squarederror',
                        n_estimators=500)

preprocessor = ColumnTransformer(
    transformers = [
        ('num', num_transformer, num_cols),
        ('ord', ord_transformer, ord_cols),
        ('oh', oh_transformer, oh_cols)
    ],
    remainder = 'passthrough'
)

pipeline = Pipeline(
    steps = [
        ('preprocessor', preprocessor),
        ('predict', my_model)
    ]
)

X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns=['Price']), train['Price'], test_size=0.2, random_state=42)

pipeline.fit(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


XGBoostError: [03:17:25] /workspace/src/objective/init_estimation.h:16: Check failed: info.labels.Shape(0) == info.num_row_ (0 vs. 240000) : Invalid shape of labels.
Stack trace:
  [bt] (0) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x1ba24e) [0x7c0e8f33724e]
  [bt] (1) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x55fbd4) [0x7c0e8f6dcbd4]
  [bt] (2) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x4900d1) [0x7c0e8f60d0d1]
  [bt] (3) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x4c0716) [0x7c0e8f63d716]
  [bt] (4) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x4c5274) [0x7c0e8f642274]
  [bt] (5) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x70) [0x7c0e8f2deef0]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7c0f0671fe2e]
  [bt] (7) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7c0f0671c493]
  [bt] (8) /usr/lib/python3.10/

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953
...,...,...,...,...,...,...,...,...,...
499995,Adidas,Canvas,Large,2.0,Yes,No,Messenger,Red,7.383498
499996,Nike,Polyester,Small,9.0,No,Yes,Messenger,Pink,6.058394
499997,Jansport,Nylon,Small,9.0,No,Yes,Tote,Green,26.890163
